# Algebras and Bars

## Algebra

An algebra  is a pair $(A, \varpi)$ where

1. $A$ is a chain complex.
2. $\varpi$ (the *product*) is a chain complex morphism 
$$\varpi:  A \otimes A  \rightarrow A.$$

These components must satisfy the usual structural properties of the algebras.

### Implementation of an algebra

An algebra  is represented as an instance of the CLOS class `ALGEBRA`, subclass of the ` CHAIN COMPLEX` class. The definition of this new class is simply:

```commonlisp
(DEFCLASS ALGEBRA (chain-complex)
    ((aprd :type morphism :initarg :aprd :reader aprd1)))   ;; product
```

So, this new class inherits the slots of the `CHAIN COMPLEX` class and has a slot of its own, namely `aprd`, an  object of type `MORPHISM` representing the product. Like in a coalgebra, the user will note the important following fact that, **once an algebra has been defined, one may use on it any function or method applicable to a chain complex**.

### The function `build-algb`

To facilitate the construction of instances of the `ALGEBRA` class, the software **Kenzo** provides the function `build-algb`:

```commonlisp
build-algb :cmpr cmpr :basis basis :bsgn bsgn :intr-dffr intr-dffr
           :dffr-strt dffr-strt :intr-aprd intr-aprd
           :aprd-strt aprd-strt :orgn orgn
```

defined with keyword parameters. The returned value is an instance of type `ALGEBRA`. 
The keyword arguments are:

- *cmpr*, a comparison function for generators.
- *basis*, the basis function for the underlying chain complex.
- *bsgn*, the  base point, a generator of any type.
- *intr-dffr*, the differential lisp function for the chain complex.
- *dffr-strt*, the strategy (`:gnrt` or `:cmbn`) for the differential.
- *intr-aprd*, the lisp function for the product of the algebra.
- *aprd-strt*, the strategy (`:gnrt` or `:cmbn`) for the coproduct.
- *orgn*, an adequate comment list.

The function `build-algb` calls  the function `build-chcm` and with the help of the function `build-mrph` builds the product morphism of degree 0 to settle the
slot `aprd` of the instance. According to the definition, the source slot of the product morphism is the tensor product of this chain complex with itself and the target slot is the underlying chain complex. An identification number $n$ (slot `idnm`) is assigned to this Kenzo object and the `ALGEBRA` instance is  pushed  onto the list `*algb-list*`. 
The  associated printing method prints a string like `[K`*n* `Algebra]`.

### Miscellaneous functions and macros for algebras

`cat-init` *[Function]*
> Clear among others, the list `*algb-list*`, list of user created algebras  and reset the global counter to 1.

`algb` *n* *[Function]*
> Retrieve in the list `*algb-list*` the algebra instance whose identification is $n$. If it does not exist, return `NIL`.

`aprd` *algb* `&rest` *[Macro]*
> Versatile macro relative to the product of an algebra. The first argument is an `ALGEBRA`
instance. With only one argument, this macro returns the product morphism of the algebra.
With more arguments, it applies the product morphism on a combination (`(aprd` *algb cmbn*`)`)
or on a pair *degree, tensor-product* (`(aprd` *algb degr tensor-product*`)`).

`change-chcm-to-algb` *chcm* `:intr-aprd` *intr-aprd* `:aprd-strt` *aprd-strt* `:orgn` *orgn* *[Function]*
> Build an algebra instance from the already created chain complex *chcm*. The user must give via key parameters a lisp function for the product, its strategy and a comment list.

## Hopf Algebra

In **Kenzo**, a Hopf Algebra is an instance of the class `HOPF-ALGEBRA`, the definition of which being simply:

```commonlisp
(DEFCLASS HOPF-ALGEBRA (coalgebra algebra)
  ())
```

So, this class (multi-) inherits the slots of the `COALGEBRA` and `ALGEBRA` classes.
The  associated printing method prints a string like `[K`*n* `Hopf-Algebra]`, and the function `hopf` may be used to retrieve the Hopf instance, in the list `*hopf-list*`.

### Example of algebra

We shall see later important examples of algebras. Let us content ourself for the moment
to define the trivial algebra on a given chain complex *having only one generator in degree*  0}.  If  this chain complex *chcm* already exists, we use simply the function `change-chcm-to-algb` and we have to provide for the keyword `:intr-aprd`, a lisp function for the product morphism. We recall that the argument of the product in the algebra is a tensor product.
This function must obey the rule of the multiplication by the unit and otherwise will return a null combination of degree the sum of the two degrees in the tensor product.

In [1]:
(defun trivial-algebra (chcm)
   (change-chcm-to-algb  
      chcm
      :intr-aprd #'(lambda (degr tnpr)
                           (declare (ignore degr))
                           (with-tnpr (degr1 gnrt1 degr2 gnrt2) tnpr
                               (if (zerop degr1)
                                   (cmbn degr2 1 gnrt2)
                                   (if (zerop degr2)
                                       (cmbn degr1 1 gnrt1)
                                       (zero-cmbn (+ degr1 degr2))))))
      :aprd-strt :gnrt
      :orgn `(trivial-algebra ,chcm)))

TRIVIAL-ALGEBRA

A good example of a chain complex to be given as argument to the previous function, is the simplicial set `smp-deltab2` that we used as simple example in the coalgebra chapter:

In [2]:
(def smp-deltab2
   (build-smst
        :cmpr #'(lambda(gsm1 gsm2)
                       (if(rest gsm1) (l-cmpr gsm1 gsm2) :equal))
        :basis :locally-effective
        :bspn '(0)
        :face #'(lambda (i dmn gsm)
                        (case dmn
                         (0 (error "No face in dimension 0"))
                         (1 (error "No non-degenerate simplex in dimension 1"))
                         (2 (absm 1 '(0)))
                         (otherwise (absm 0 (append (subseq gsm 0 i)
                                                    (subseq gsm (1+ i))))) ))
        :orgn '(simple-deltab2)))

SIMPLE-STYLE-WARNING: 
  #<SB-INT:SIMPLE-STYLE-WARNING "undefined ~(~A~): ~S" {100530D143}>


[K1 Simplicial-Set]

In [3]:
(trivial-algebra smp-deltab2)

[K1 Algebra]

Note that now `smp-deltab2` is of type `ALGEBRA` and has kept its Kenzo numbering.

In [4]:
smp-deltab2

[K1 Algebra]

In [5]:
(inspect smp-deltab2)


The object is a STANDARD-OBJECT of type CAT::ALGEBRA.
0. CMPR: #<FUNCTION (LAMBDA (GSM1 GSM2) :IN "/root/cl-jupyter-master/cl-jupyter.lisp") {100521C55B}>
1. BASIS: :LOCALLY-EFFECTIVE
2. BSGN: (0)
3. DFFR: [K2 Morphism (degree -1): K1 -> K1]
4. GRMD: [K1 Algebra]
5. EFHM: "unbound"
6. IDNM: 1
7. ORGN: (SIMPLE-DELTAB2)
8. APRD: [K6 Morphism (degree 0): K3 -> K1]

> 

NIL

Let us test now the product in this trivial algebra:

In [6]:
(aprd smp-deltab2 3 (tnpr 0 '(0) 3 '(1 2 3 4)))


----------------------------------------------------------------------{CMBN 3}
<1 * (1 2 3 4)>
------------------------------------------------------------------------------


In [7]:
(aprd smp-deltab2 3 (tnpr  3 '(1 2 3 4) 0 '(0)))


----------------------------------------------------------------------{CMBN 3}
<1 * (1 2 3 4)>
------------------------------------------------------------------------------


In [8]:
(aprd smp-deltab2 5 (tnpr 2 '(0 1 2) 3 '(6 7 8 9)))


----------------------------------------------------------------------{CMBN 5}
------------------------------------------------------------------------------


## The Bar construction

Let $\cal A$ be an associative algebra, assumed connected and  ${\cal A}_0 \simeq \mathbb{Z}$. 
Furthermore let us suppose that ${\cal A}$ is a free $\mathbb{Z}$-module. Then it is possible 
to define a chain complex, $Bar^{\cal A}(\mathbb{Z}, \mathbb{Z})$ - simply denoted here by $Bar({\cal A})$ - 
whose $n$-th component is the free $\mathbb{Z}$-module generated by the "bars":

$$ [Bar({\cal A})]_n = \{ [g_1 \mid g_2 \mid \ldots \mid g_k] \}, \quad \sum_{j=1}^{k} [deg(g_j)+1] =n. $$

The object denoted $[g_1 \mid g_2 \mid \ldots \mid g_k]$ 
with $\sum_{j=1}^{k} [deg(g_j)+1] =n$, is traditionally called a *bar* and is in fact,  an element of the $n$-th iterated suspension of ${\cal A}^{\otimes n}$. The integer $n$ is the *total degree*. The structure of the bar of the algebra ${\cal A}$ is recalled in the following figure, where $\bar{\cal A}$ is ${\cal A}$ without its component of degree 0. In the vertical sense, we have the *tensorial degree*, whereas in the horizontal one, we have the *simplicial degree*. The *total degree* $n$ is the sum of both degrees.

$$
\begin{array}{|ccccc}
  \vdots & \vdots & \vdots &\vdots & \vdots \\
  0 & \bar{\cal A}_3 & (\bar{\cal A} \otimes \bar{\cal A})_3 & 
  (\bar{\cal A}\otimes \bar{\cal A}\otimes\bar{\cal A})_3 & \cdots \\
  0 & \bar{\cal A}_2 & (\bar{\cal A}\otimes\bar{\cal A})_2 & 0 & \cdots \\
  0 & \bar{\cal A}_1 & 0 & 0 & \cdots \\
  \mathbb{Z} & 0 & 0 & 0 & \cdots \\ \hline
    &   &   &   &    \\
  \bar{\cal A}^{\otimes 0} & \bar{\cal A}^{\otimes 1} & \bar{\cal A}^{\otimes 2} &
  \bar{\cal A}^{\otimes 3} & \cdots 
\end{array}
$$

### Representation of a bar object

An elementary bar object - not to be confused with the chain complex $Bar({\cal A})$ - is represented in `Kenzo` by a lisp object of the form:

`(:ABAR`  ($i_1\, . \, a_1$) \ldots ($i_k\, . \, a_k$)`)`

where the $i_j$ are the degrees *in the bar chain complex* of the generators $a_j$.
In the original algebra $\cal A$, the generators $a_j$ had the degree $i_j-1$. The corresponding type is `ABAR`. The function  building such an object is also called `abar`

`abar` *degr1 gnr1 degr2 gnr2 ... degrk gnrk* *[Function]*
> Construct an elementary bar object, i.e. a "suspended tensorial product" of degree $\sum degr_i$. The sequence of pairs $\lbrace degr_i,\,  gnr_i \rbrace$ has an undefinite length and  may be void. In this case, the bar is the null bar object, also located in the system through the constant `+null-abar+`. The function `abar` accepts also as unique argument a list of the form `(`*degr1 gnr1 degr2 gnr2 ... degrk gnrk*`)`.

`abar-p` *object* *[Function]*
> Test if *object* is an elementary bar object.

The associated printing  method prints the object under the form:

`<<Abar [`$i_1\quad a_1$ `]` ... `[` $i_k\quad a_k$ `] >>`

#### Examples

These simple commands show the two different ways to create bar objects
from elements of an algebra (or a chain complex).

In [9]:
(abar 2 'a 3 'b 5 'c)

<<Abar[2 A][3 B][5 C]>>

In [10]:
(abar '(2 a 3 b 5 c))

<<Abar[2 A][3 B][5 C]>>

In [11]:
(abar)

<<Abar>>

### Definition of the chain complex Bar

The definition of the differential in the bar chain complex, will be done in three steps. This is better understood,
if we consider the following diagram.

$$
\begin{aligned}
       &             & \downarrow\quad  &  & \downarrow\quad \cr
\cdots\quad & \leftarrow & ({\bar{\cal A}}_*^{\otimes p})_q & \qquad\buildrel {d_h} \over \longleftarrow 
                     & ({\bar{\cal A}}_*^{\otimes {p+1}})_q  & \quad\leftarrow & \cdots \cr
       &             &  \downarrow {\scriptstyle d}_v  &  &  \downarrow {\scriptstyle d}_v \cr
\cdots\quad & \leftarrow &({\bar{\cal A}}_*^{\otimes p})_{q-1} & \qquad\buildrel {d_h} \over \longleftarrow 
                     & ({\bar{\cal A}}_*^{\otimes {p+1}})_{q-1}  & \quad\leftarrow & \cdots \cr 
       &             & \downarrow\quad  &  & \downarrow\quad \cr
\end{aligned}
$$

- A chain complex called `vertical-bar` is defined with the *vertical differential* $d_v$. In this case, only the  underlying chain complex structure is used.
- A horizontal differential $d_h$ is defined. This uses  the product structure.
- The final chain complex `bar` is created with differential $d_v+d_h$.

#### Definition of the vertical bar

To define the vertical bar chain complex from the chain complex ${\cal A}$, the following functions are provided:

`bar-cmpr` *cmpr* *[Function]*
> From the comparison function *cmpr*, build a comparison function for objects of type `abar`.

`bar-basis` *basis* *[Function]*
> From the function *basis* of a  chain complex ${\cal A}$, build a basis function for the vertical bar chain complex defined on ${\cal A}$. In dimension 0, there is only one basis element, namely the null abar object. If ${\cal A}$ is locally effective, this function returns the symbol `:locally-effective`.

`bar-intr-vrtc-dffr` *dffr* *[Function]*
> From the `lisp` function *dffr* of a chain complex  ${\cal A}$, build a lisp function for the differential of the vertical bar chain complex, according to the formula:
$$d_v [a_1 \mid \cdots \mid a_n]=
-\sum_{i=1}^n{(-1)^{\sum_{j=1}^{i-1}{\mid a_j \mid}}}
 {[a_1 \mid \cdots \mid a_{i-1} \mid {\bf da_i} \mid a_{i+1} \mid \cdots \mid a_n]},$$
where $da_i$ is the differential of the generator $a_i$ in the original chain complex (function *dffr*) and $\mid a_i \mid$ is the degree of the generator $a_i$ *in the bar chain complex*.
In general, the differential $da_i$ is a sum of  objects. So, applying the distributive law, the right member is a sum of bar objects, represented by a combination.

`vrtc-bar` *chcm* *[Method]*
> Build the vertical bar chain complex from the underlying chain complex *chcm*. This is simply done by the call to `build-chcm`:
``lisp
         (build-chcm
            :cmpr  (bar-cmpr cmpr)
            :basis (bar-basis basis)
            :bsgn +null-abar+
            :intr-dffr (bar-intr-vrtc-dffr dffr)
            :strt :gnrt
            :orgn `(vrtc-bar ,chcm))
``
where, *cmpr*, *basis* and *dffr* are extracted from the chain complex *chcm*. Note that the base generator is the null bar object.

#### Examples

First, let us test the functions `bar-cmpr` and `bar-basis`. With the first one, applied to the elementary comparison function `s-cmpr`, we create a comparison function suitable for algebras where the generators are symbols.

In [12]:
(def cmpr-for-bar (bar-cmpr #'s-cmpr))

#<CLOSURE (FLET CAT::RSLT :IN BAR-CMPR) {1005AE1E4B}>

In [13]:
(funcall cmpr-for-bar (abar)(abar))

:EQUAL

In [14]:
(funcall cmpr-for-bar (abar 3 'a 7 'q) (abar))

:GREATER

In [15]:
(funcall cmpr-for-bar (abar '(1 x 2 y 3 z) )(abar 1 'x 2 'y 3 'z))

:EQUAL

Let us suppose now that the function `simple-basis` is the basis function for a certain algebra (chain complex): in degree $i$, the basis is only the list $(a_i)$.

In [16]:
(def simple-basis #'(lambda(degr) 
    (list (intern(format nil "A~D" degr)))))

#<FUNCTION (LAMBDA (DEGR) :IN "/root/cl-jupyter-master/cl-jupyter.lisp") {1005C9C5CB}>

In [17]:
(funcall simple-basis 5)

(A5)

The function `bar-basis` creates the function for the basis of the corresponding bar chain complex. Note that when this new  function is applied, it returns a priori the null abar in dimension 0 and a null basis in dimension 1. Note also that the basis elements of the underlying algebra (the $a_i$'s) are suspended.

In [18]:
(def basis-for-bar (bar-basis simple-basis))

#<CLOSURE (FLET CAT::RSLT :IN BAR-BASIS) {1005D5F11B}>

In [19]:
(dotimes (i 7)
   (format t "~%~D ~A" i (funcall basis-for-bar i)))


0 (<<Abar>>)
1 NIL
2 (<<Abar[2 A1]>>)
3 (<<Abar[3 A2]>>)
4 (<<Abar[4 A3]>> <<Abar[2 A1][2 A1]>>)
5 (<<Abar[5 A4]>> <<Abar[2 A1][3 A2]>> <<Abar[3 A2][2 A1]>>)
6 (<<Abar[6 A5]>> <<Abar[2 A1][4 A3]>> <<Abar[3 A2][3 A2]>> <<Abar[4 A3][2 A1]>> <<Abar[2 A1][2 A1][2 A1]>>)

NIL

To test the vertical differential, let us take the simplicial set `smp-deltab2` which is also a chain complex. We may use it for testing the function `vrtc-bar` which defines a chain complex independently of the product of the algebra. We recall that, in dimension $n$, the elements of the chain complex `smp-deltab2` are represented by lists of increasing $n+1$  integers. So, in the abar objects, the degree of such elements are $n+1$.

In [20]:
(def vrt-bar (vrtc-bar smp-deltab2))

[K7 Chain-Complex]

In [21]:
(? vrt-bar 9 (abar 4 '(0 1 2 3) 5 '(4 5 6 7 8)))


----------------------------------------------------------------------{CMBN 8}
<1 * <<Abar[3 (0 1 2)][5 (4 5 6 7 8)]>>>
<-1 * <<Abar[3 (0 1 3)][5 (4 5 6 7 8)]>>>
<1 * <<Abar[3 (0 2 3)][5 (4 5 6 7 8)]>>>
<-1 * <<Abar[3 (1 2 3)][5 (4 5 6 7 8)]>>>
<-1 * <<Abar[4 (0 1 2 3)][4 (4 5 6 7)]>>>
<1 * <<Abar[4 (0 1 2 3)][4 (4 5 6 8)]>>>
<-1 * <<Abar[4 (0 1 2 3)][4 (4 5 7 8)]>>>
<1 * <<Abar[4 (0 1 2 3)][4 (4 6 7 8)]>>>
<-1 * <<Abar[4 (0 1 2 3)][4 (5 6 7 8)]>>>
------------------------------------------------------------------------------


In [22]:
(? vrt-bar *)


----------------------------------------------------------------------{CMBN 7}
------------------------------------------------------------------------------


#### Definition of the horizontal differential

So far, the algebra structure has not been used but now it will be the main ingredient.
The canonical "horizontal"  differential $d_h$ is defined as follows:

$$d_h:({\bar{\cal A}}_*^{\otimes p})_q \longrightarrow ({\bar{\cal A}}_*^{\otimes {p-1}})_q,$$

$$d_h [a_1 \mid \cdots \mid a_k] = \sum_{i=2}^n{(-1)^{\sum_{j=1}^{i-1}{\mid a_j \mid}}}
      [a_1 \mid \cdots \mid a_{i-2} \mid {\bf a_{i-1}a_i} \mid a_{i+1} \mid \cdots \mid a_k]$$

where the product in the algebra is noted simply by the concatenation and $\mid a_i \mid$
is the degree of $a_i$ in $Bar({\cal A})$. In `Kenzo`, two functions are designed for building the horizontal differential:

`bar-intr-hrzn-dffr` *aprd* *[Function]*
> From the product morphism *aprd*, build the lisp function with 2 arguments: a degree and an `abar` object (i.e. a generator) realizing the algorithm for $d_h$.

`bar-hrzn-dffr` *algb* *[Function]*
Build the horizontal differential morphism, using the slot product *aprd* of the algebra *algb* and the previous function. Note that when called, this function creates, if not already created, the vertical bar chain complex on  * algb*:
```commonlisp
           (build-mrph
             :sorc (vrtc-bar algb) 
             :trgt (vrtc-bar algb) 
             :degr -1
             :intr (bar-intr-hrzn-dffr aprd) 
             :strt :gnrt
             :orgn `(bar-hrzn-dffr ,algb))
```

#### Final definition of the bar of an algebra

To define completely the bar chain complex, we have to provide the lisp function for the differential $d_v + d_h$ and, once created the vertical bar chain complex, we have to call the building function `build-chcm` with adequate arguments.

`bar-intr-dffr` *vrtc-dffr hrzn-dffr* *[Function]*
> Define the lisp function for the differential $d_v+d_h$ from the two morphisms *vrtc-dffr* and *hrzn-dffr*. For efficiency reasons, the implementor has chosen to define this new function rather than to use simply
the addition of two morphisms.

`bar` *algebra* *[Method]*
> Build first the vertical bar chain complex on the algebra. Then return a chain complex with the same slots as the vertical bar chain complex but with new `:intr-dffr` and `:orgn` slots, as shown in the following definition:
```commonlisp
            (defmethod BAR ((algebra algebra))
                (let ((vrtc-bar (vrtc-bar algebra))
                      (bar-hrzn-dffr (bar-hrzn-dffr algebra)))
                (declare (type chain-complex vrtc-abr hrzn-bar))
                (the chain-complex
                    (build-chcm
                       :cmpr (cmpr vrtc-bar)
                       :basis (basis vrtc-bar)
                       :bsgn +null-abar+
                       :intr-dffr (bar-intr-dffr (dffr vrtc-bar) 
                                                   bar-hrzn-dffr)
                       :strt :gnrt
                       :orgn `(add ,vrtc-bar ,bar-hrzn-dffr)))))
```

We postpone the examples for the bar of an algebra, up to the chapter on simplicial groups where we shall  build interesting examples of algebras.

## Other functions for the bar construction

The vertical bar construction is *natural* and therefore defines a functor. Consequently, the `Kenzo` program contains the following methods.

`vrtc-bar` *mrph* *[Method]*
> From the morphism *mrph*, build a new morphism between the vertical-bar of the source of *mrph* and the vertical bar of the target of *mrph*, the morphism itself being induced in a natural way by the underlying morphism.

`vrtc-bar` *rdct* *[Method]*
> From the reduction *rdct*, build a new reduction, by applying the vertical bar method to the functions $f$ and $g$. The new homotopy cannot be obtained so simply, being a function of the $f$, $g$ and $h$ of *rdct*. Nevertheless, the source and target of the new homotopy are the vertical bar of the underlying homotopy.

`vrtc-bar` *hmeq* *[Method]*
> Build a homotopy equivalence by applying the vertical bar construction to the left and right reductions of the homotopy equivalence *hmeq*.

`bar` *hmeq* *[Method]*
> Build a  homotopy equivalence where the bar construction is applied to the underlying homotopy equivalence in the following way. First this construction is limited to the case where the left bottom chain complex of *hmeq* is an ** algebra**. Then the vertical bar construction is applied to *hmeq* and the horizontal differential of the left bottom chain complex is propagated upon the new homotopy equivalence with the help of the method `add`. In other words,
starting from the left bottom chain complex, firstly the easy pertubation lemma is applied to obtain a new differential on the top chain complex; then the basic perturbation lemma is applied to the right reduction. If *hmeq* is a trivial homotopy equivalence, this function returns a trivial one, built on the bar of the left bottom chain complex of * hmeq*.

### Lisp files concerned in this chapter

`algebras.lisp`, `bar.lisp`.

[`classes.lisp`, `macros.lisp`, `various.lisp`].